In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dotenv import load_dotenv
import datetime as dt
import talib as ta
from tradingpatterns.tradingpatterns import detect_double_top_bottom
from securities_load.securities.postgresql_database_functions import sqlalchemy_engine
from securities_load.securities.securities_table_functions import retrieve_ohlcv_from_to

pd.__version__  # for the record

%matplotlib inline

load_dotenv()
engine = sqlalchemy_engine()

In [28]:
# Get the data and have a look at it
exchange_code = "XNAS" # XASX, XNYS, XNAS
ticker = 'AAPL'
num_of_years = 1.5
start_date = dt.datetime.now() - dt.timedelta(int(365.25 * num_of_years))
start_string_date = dt.datetime.strftime(start_date, "%Y-%m-%d")
end_date = dt.datetime.now()
end_string_date = dt.datetime.strftime(end_date, "%Y-%m-%d")
end_string_datetime = dt.datetime.strftime(end_date, "%Y-%m-%d 00:00:00")
df = retrieve_ohlcv_from_to(engine, exchange_code=exchange_code, ticker=ticker, start_date=start_string_date, end_date=end_string_date)
short_ema_period = 8
long_ema_period = 21
sma_period = 10
rsi_period = 14
interval = 'daily'
acceleration=0.02
maximum=0.2

Exchange_id is: 2


In [29]:
df['sma_daily'] = ta.SMA(df['close'], timeperiod=sma_period) # type: ignore
df['sar_daily'] = ta.SAR(df['high'], df['low'], acceleration, maximum) # type: ignore
df['macd_line'], df['signal_line'], df['histogram'] = ta.MACD(df['close']) # type: ignore
df['ema_daily_short'] = ta.EMA(df['close'], timeperiod=short_ema_period) # type: ignore
df['ema_daily_long'] = ta.EMA(df['close'], timeperiod=long_ema_period) # type: ignore
df['rsi_daily'] = ta.RSI(df['close'], timeperiod=rsi_period) # type: ignore

In [30]:
df_uppercase = df[['date', 'open', 'high', 'low', 'close', 'volume']]
df_uppercase.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df_double = detect_double_top_bottom(df_uppercase)
df_double[['High','Low','high_roll_max', 'double_pattern']].head(20)
# df_new = df_double.dropna()
# df_new.tail(20)

/home/ubuntuuser/miniconda3/envs/analysis/lib/python3.12/site-packages/tradingpatterns/tradingpatterns.py:118: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/ubuntuuser/miniconda3/envs/analysis/lib/python3.12/site-packages/tradingpatterns/tradingpatterns.py:119: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/ubuntuuser/miniconda3/envs/analysis/lib/python3.12/site-packages/tradingpatterns/tradingpatterns.py:127: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataF

,High,Low,high_roll_max,double_pattern
Datetime,,,,
2023-08-07,183.1300,177.350,NaN,NaN
2023-08-08,180.2700,177.580,NaN,NaN
2023-08-09,180.9300,177.010,183.1300,NaN
2023-08-10,180.7500,177.600,180.9300,NaN
2023-08-11,178.6200,176.550,180.9300,Double Top
2023-08-14,179.6900,177.305,180.7500,Double Bottom
2023-08-15,179.4800,177.050,179.6900,NaN
2023-08-16,178.5400,176.500,179.6900,NaN
2023-08-17,177.5054,173.480,179.4800,NaN


In [31]:
## Volume
# From our Dataframe take only the rows where the Close > Open
# save it in different Dataframe, these should be green
df_double['green_volume'] = df_double[df_double['Close'] >= df_double['Open']]['Volume']
# Same for Close < Open, these are red candles/bars
df_double['red_volume'] = df_double[df_double['Close'] < df_double['Open']]['Volume']
# df.head()
df_double.tail()

/tmp/ipykernel_4803/691117220.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_4803/691117220.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Open,High,Low,Close,Volume,high_roll_max,low_roll_min,double_pattern,green_volume,red_volume
Datetime,,,,,,,,,,,
2025-01-27,2025-01-27,224.020,232.150,223.98,229.86,90457014,232.15,221.41,NaN,90457014.0,NaN
2025-01-28,2025-01-28,230.850,240.190,230.81,238.26,72794131,240.19,221.41,NaN,72794131.0,NaN
2025-01-29,2025-01-29,234.120,239.855,234.01,239.36,43732107,240.19,223.98,NaN,43732107.0,NaN
2025-01-30,2025-01-30,238.665,240.790,237.21,237.59,53505269,240.79,230.81,NaN,NaN,53505269.0
2025-01-31,2025-01-31,247.190,247.190,233.44,236.00,90132405,247.19,233.44,NaN,NaN,90132405.0


In [32]:
# Individual Plot Elements:

## OHLC Candlestick Chart
trace_candles = go.Candlestick(x=df_double.index, # df_daily.index stores dates (x-axis)
        open=df_double.Open, # Open for OHLC candlesticks
        high=df_double.High, # High for OHLC candlesticks
        low=df_double.Low, # Low for OHLC candlesticks
        close=df_double.Close, # Close for OHLC candlesticks
        name='Candlestick') # Naming this to Candlestick for legend on the side of plot

# # Open and Close markers
# d = 1 # Marker will be placed d position points above or below daily open/close valu, respectively.
# df_daily["marker"] = np.where(df_daily["open"] < df_daily["close"], df_daily["high"] + d, df_daily["low"] - d)
# df_daily["Symbol"] = np.where(df_daily["open"] < df_daily["close"], "triangle-up", "triangle-down") # triangle up for + day, triangle down for - day
# df_daily["Color"] = np.where(df_daily["open"] < df_daily["close"], "green", "red") # defining green positive change and red for negative daily change

# # Arrows corresponding to daily increasing/decreasing values
# trace_arrow = go.Scatter(x=list(df_daily.index),
#                             y=list(df_daily.marker),
#                             mode='markers',
#                             name='Markers',
#                             marker=go.scatter.Marker(size=8,
#                                                     symbol=df_daily["Symbol"],
#                                                     color=df_daily["Color"]))

# 8 Day EMA over 21 Day EMA:

# EMA
trace_ema8 = go.Scatter(x=list(df.index),
                        y=list(df['ema_daily_short']),
                        name='8 Day EMA',
                        line=dict(color='#E45756', # Define color for line
                                width=1, # Define width for line
                                dash='dot')) # Define dash (I want my line to be dotted)

trace_ema21 = go.Scatter(x=list(df.index),
                        y=list(df['ema_daily_long']),
                        name='21 Day EMA',
                        line=dict(color='#4C78A8', # Define color for line
                                width=1, # Define width for line
                                dash='dot')) # Define dash (I want my line to be dotted)

## SMA
trace_sma = go.Scatter(x=list(df.index),
                        y=list(df['sma_daily']),
                        name=str(sma_period)+' Day SMA',
                        line=dict(color='#E45756',
                                width=1,
                                dash='dot'))

## Volume
# From our Dataframe take only the rows where the Close > Open
# save it in different Dataframe, these should be green
# green_volume_df = df[df['close'] > df['open']]
# # Same for Close < Open, these are red candles/bars
# red_volume_df = df[df['close'] < df['open']]

# Plot the red bars and green bars in the second subplot
# trace_volume_red = go.Bar(x=red_volume_df.index, y=red_volume_df.volume, showlegend=False, marker_color='#ef5350')
# trace_volume_green = go.Bar(x=green_volume_df.index, y=green_volume_df.volume, showlegend=False, marker_color='#26a69a')
trace_volume_red = go.Bar(x=list(df_double.index),
                        y=list(df_double['red_volume']),
                        # showlegend=False,
                        name='Volume Down Day',
                        marker=dict(color='red'),
                        )

trace_volume_green = go.Bar(x=list(df_double.index),
                        y=list(df_double['green_volume']),
                        # showlegend=False,
                        name='Volume Up Day',
                        marker=dict(color='green'),
                        )

# MACD Histogram
trace_macd_hist = go.Bar(x=list(df.index),
                        y=list(df['histogram']),
                        name='MACD Histogram',
                        marker=dict(color='gray'),
                        yaxis='y3',
                        legendgroup='three')

## MACD Line
trace_macd = go.Scatter(x=list(df.index),
                        y=list(df['macd_line']),
                        name='MACD',
                        line=dict(color='black', width=1.5),  # red
                        yaxis='y3',
                        legendgroup='three')

## MACD Signal Line
trace_macd_signal = go.Scatter(x=list(df.index),
                        y=list(df['signal_line']),
                        name='Signal',
                        line=dict(color='red', width=1.5),  # plum
                        yaxis='y3',
                        legendgroup='three')

## RSI
trace_rsi = go.Scatter(x=list(df.index),
                        y=list(df['rsi_daily']),
                        mode='lines',
                        name='RSI',
                        line=dict(color='black',
                                width=1.5),
                        yaxis='y4',
                        legendgroup='four')

# RSI Overbought
trace_rsi_70 = go.Scatter(mode='lines',
                        x=[min(df.index), max(df.index)],
                        y=[70, 70],
                        name='Overbought > 70%',
                        line=dict(color='green',
                                width=0.5,
                                dash='dot'),
                        yaxis='y4',
                        legendgroup='four')

# RSI Oversold
trace_rsi_30 = go.Scatter(mode='lines',
                        x=[min(df.index), max(df.index)],
                        y=[30, 30],
                        name='Oversold < 30%',
                        line=dict(color='red',
                                width=0.5,
                                dash='dot'),
                        yaxis='y4',
                        legendgroup='four')

# RSI Center Line
trace_rsi_50 = go.Scatter(mode='lines',
                        x=[min(df.index), max(df.index)],
                        y=[50, 50],
                        line=dict(color='gray',
                                width=0.5,
                                dash='dashdot'),
                        name='50%',
                        yaxis='y4',
                        legendgroup='four')

In [37]:
## Plotting Layout
layout = go.Layout(
        xaxis=dict(titlefont=dict(color='yellowgreen'), # Color of our X-axis Title
                tickfont=dict(color='yellowgreen'), # Color of ticks on X-axis
                linewidth=1, # Width of x-axis
                linecolor='black', # Line color of x-axis
                gridwidth=1, # gridwidth on x-axis marks
                gridcolor='rgb(204,204,204)', # grid color
                # Define ranges to view data. I chose 3 months, 6 months, 1 year, and year to date
                rangeselector=dict(
                        buttons=(dict(count=6, label='6 mo', step='month', stepmode='backward'),
                                dict(count=1, label='YTD', step='year', stepmode='todate'),
                                dict(count=1, label='1 yr', step='year', stepmode='backward'),
                                dict(step='all')))),
        xaxis4=dict(titlefont=dict(color='yellowgreen'), # Color of our X-axis Title
                tickfont=dict(color='yellowgreen'), # Color of ticks on X-axis
                title='Date',
                linewidth=1, # Width of x-axis
                linecolor='black', # Line color of x-axis
                gridwidth=1, # gridwidth on x-axis marks
                gridcolor='rgb(204,204,204)'), # grid color)
                # Define different y-axes for each of our plots: daily, volume, MACD, and RSI -- hence 4 y-axes
        yaxis=dict(fixedrange=False, title='Price',
                titlefont=dict(color='yellowgreen'),
                tickfont=dict(color='yellowgreen'),
                linecolor='black',
                mirror='all',
                gridwidth=1,
                gridcolor='rgb(204,204,204)'),
        yaxis2=dict(fixedrange=False, title='Volume',
                titlefont=dict(color='yellowgreen'),
                tickfont=dict(color='yellowgreen'),
                linecolor='black',
                mirror='all',
                gridwidth=1,
                gridcolor='rgb(204,204,204)'),
        yaxis3=dict(fixedrange=False, title='MACD',
                titlefont=dict(color='yellowgreen'),
                tickfont=dict(color='yellowgreen'),
                linecolor='black',
                constraintoward='center',  # might not be necessary
                mirror='all',
                gridwidth=1,
                gridcolor='rgb(204,204,204)'),
        yaxis4=dict(range=[10, 90], title='RSI',
                tick0=10, dtick=20,
                titlefont=dict(color='yellowgreen'),
                tickfont=dict(color='yellowgreen'),
                linecolor='black',
                mirror='all',
                gridwidth=1,
                gridcolor='rgb(204,204,204)'),
        legend=dict(font=dict(color='yellowgreen')),
        title=(ticker + ' Daily Data'),
        titlefont=dict(color='yellowgreen'), # Give our plot a title
        title_x=0.5, # Center our title
        paper_bgcolor='rgba(37,37,37,0)',  # Background color of main background
        plot_bgcolor='rgb(226,238,245)',  # Background color of plot
        height=900, # overall height of plot
        margin=dict(l=60, r=20, t=50, b=5) # define margins: left, right, top, and bottom
        )

In [39]:
figure = make_subplots(rows=4, cols=1, shared_xaxes=True, row_heights=[0.55, 0.15, 0.15, 0.15], vertical_spacing=0.01)

figure.add_trace(trace_candles, row=1, col=1)
figure.add_trace(trace_ema8, row=1, col=1)
figure.add_trace(trace_ema21, row=1, col=1)
figure.add_trace(trace_sma, row=1, col=1)

df_notnan = df_double[df_double['double_pattern'].notnull()]
df_notnan.head()
for idx in range(len(df_notnan)):
    figure.add_annotation(
        x=df_notnan.index[idx],
        y=df_notnan.iloc[idx]['High'], # + (df_notnan['High'] - df_notnan['Low']) * 0.05,  # Adjust the offset to be 10% of the candle height
                    # y=df.iloc[idx]['high'],  # Adjust the offset to be 10% of the candle height
        text=df_notnan.iloc[idx]['double_pattern'],
                    showarrow=True,
                    arrowhead=1,
                    # textangle=90,
                    yshift=1  # Optional: Shift the text slightly down if needed
                )

figure.add_trace(trace_volume_red, row=2, col=1)
figure.add_trace(trace_volume_green, row=2, col=1)

figure.add_trace(trace_macd, row=3, col=1)
figure.add_trace(trace_macd_hist, row=3, col=1)
figure.add_trace(trace_macd_signal, row=3, col=1)

figure.add_trace(trace_rsi, row=4, col=1)
figure.add_trace(trace_rsi_70, row=4, col=1)
figure.add_trace(trace_rsi_50, row=4, col=1)
figure.add_trace(trace_rsi_30, row=4, col=1)

figure.update(layout_xaxis_rangeslider_visible=False)
figure.update_layout(layout)